# Advanced Modeling of Groundwater Flow (GW3099)
![alt text](./img/header.jpg)

# MODPATH 7 Problem 2
This is a replication of the MODPATH Problem 2 example that is described on page 12 of the modpath_7_examples.pdf file.  The results shown here should be the same as the results in the MODPATH example, however, the vertex and node numbering used here may be different from the numbering used in MODPATH, so head values may not be compared directly without some additional mapping.

## Part I. Setup Notebook

In [ ]:
# %load nbsetup.py
%matplotlib inline
from __future__ import print_function
import sys
import os
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy
import config

print(sys.version)
print('python executable: {}'.format(sys.executable))
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

## Part II. Gridgen Creation of Model Grid

In [ ]:
from flopy.utils.gridgen import Gridgen 

### Create the Base MODFLOW Grid

In [ ]:
Lx = 10000.
Ly = 10500.
nlay = 3
nrow = 21
ncol = 20
delr = Lx / ncol
delc = Ly / nrow
top = 400
botm = [220, 200, 0]

In [ ]:
ms = flopy.modflow.Modflow(rotation=-20.)
dis5 = flopy.modflow.ModflowDis(ms, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr,
                               delc=delc, top=top, botm=botm)

### Create the Gridgen Object

In [ ]:
model_name = 'mp7p2'
model_ws = os.path.join('.', model_name)
gridgen_ws = os.path.join(model_ws, 'gridgen')
g = Gridgen(dis5, model_ws=gridgen_ws)

### Refine the Grid

In [ ]:
rf0shp = os.path.join(gridgen_ws, 'rf0')
xmin = 7 * delr
xmax = 12 * delr
ymin = 8 * delc
ymax = 13 * delc
rfpoly = [[[(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax), (xmin, ymin)]]]
g.add_refinement_features(rfpoly, 'polygon', 1, range(nlay))

rf1shp = os.path.join(gridgen_ws, 'rf1')
xmin = 8 * delr
xmax = 11 * delr
ymin = 9 * delc
ymax = 12 * delc
rfpoly = [[[(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax), (xmin, ymin)]]]
g.add_refinement_features(rfpoly, 'polygon', 2, range(nlay))

rf2shp = os.path.join(gridgen_ws, 'rf2')
xmin = 9 * delr
xmax = 10 * delr
ymin = 10 * delc
ymax = 11 * delc
rfpoly = [[[(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax), (xmin, ymin)]]]
g.add_refinement_features(rfpoly, 'polygon', 3, range(nlay))

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mm = flopy.plot.ModelMap(model=ms)
mm.plot_grid()
flopy.plot.plot_shapefile(rf0shp, ax=ax, facecolor='yellow', edgecolor='none')
flopy.plot.plot_shapefile(rf1shp, ax=ax, facecolor='pink', edgecolor='none')
flopy.plot.plot_shapefile(rf2shp, ax=ax, facecolor='red', edgecolor='none')

In [ ]:
g.build(verbose=False)

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
g.plot(ax, linewidth=0.5)

### Extract the Grid Properties

In [ ]:
gridprops = g.get_gridprops_disv()
#gridprops

In [ ]:
ncpl = gridprops['ncpl']
top = gridprops['top']
botm = gridprops['botm']
nvert = gridprops['nvert']
vertices = gridprops['vertices']
cell2d = gridprops['cell2d']
cellxy = gridprops['cellxy']

## Part III. Create the Flopy Model

In [ ]:
# create simulation
sim = flopy.mf6.MFSimulation(sim_name=model_name, version='mf6', exe_name='mf6', 
                             sim_ws=model_ws)

# create tdis package
tdis_rc = [(1000.0, 1, 1.0)]
tdis = flopy.mf6.ModflowTdis(sim, pname='tdis', time_units='DAYS', 
                             perioddata=tdis_rc)

# create gwf model
gwf = flopy.mf6.ModflowGwf(sim, modelname=model_name,
                           model_nam_file='{}.nam'.format(model_name))
gwf.name_file.save_flows = True

# create iterative model solution and register the gwf model with it
ims = flopy.mf6.ModflowIms(sim, pname='ims', print_option='SUMMARY', 
                           complexity='SIMPLE', outer_hclose=1.e-5, 
                           outer_maximum=100, under_relaxation='NONE', 
                           inner_maximum=100, inner_hclose=1.e-6, 
                           rcloserecord=0.1, linear_acceleration='BICGSTAB', 
                           scaling_method='NONE', reordering_method='NONE', 
                           relaxation_factor=0.99)
sim.register_ims_package(ims, [gwf.name])

# disv
disv = flopy.mf6.ModflowGwfdisv(gwf, nlay=nlay, ncpl=ncpl, 
                                top=top, botm=botm, 
                                nvert=nvert, vertices=vertices, 
                                cell2d=cell2d)

# initial conditions
ic = flopy.mf6.ModflowGwfic(gwf, pname='ic', strt=320.)

# node property flow
npf = flopy.mf6.ModflowGwfnpf(gwf, xt3doptions=[('xt3d')], 
                              icelltype=[1,0,0], 
                              k=[50.0, 0.01, 200.0],
                              k33=[10., 0.01, 20.])

# wel
wellpoints = [(4750., 5250.)]
welcells = g.intersect(wellpoints, 'point', 0)
#welspd = flopy.mf6.ModflowGwfwel.stress_period_data.empty(gwf, maxbound=1, aux_vars=['iface'])
welspd = [[(2, icpl), -150000, 0] for icpl in welcells['nodenumber']]
wel = flopy.mf6.ModflowGwfwel(gwf, print_input=True,
                              auxiliary=[('iface', )],
                              stress_period_data=welspd)

# rch
aux = [np.ones(ncpl, dtype=np.int)*6]
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=0.005, 
                               auxiliary=[('iface',)], 
                               aux={0: [6]})
# riv
riverline = [[[(Lx - 1., Ly), (Lx - 1., 0.)]]]
rivcells = g.intersect(riverline, 'line', 0)
rivspd = [[(0, icpl), 320., 100000., 318] for icpl in rivcells['nodenumber']]
riv = flopy.mf6.ModflowGwfriv(gwf, stress_period_data=rivspd)

# output control
oc = flopy.mf6.ModflowGwfoc(gwf, pname='oc', budget_filerecord='{}.cbb'.format(model_name),
                            head_filerecord='{}.hds'.format(model_name),
                            headprintrecord=[('COLUMNS', 10, 'WIDTH', 15,
                                              'DIGITS', 6, 'GENERAL')],
                            saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
                            printrecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')])

sim.write_simulation()

## Part III. Run the MODFLOW 6 Model

In [ ]:
sim.run_simulation()

## Part IV. Import and Plot the Results

### Plot the Boundary Locations

In [ ]:
fname = os.path.join(model_ws, model_name + '.disv.grb')
grd = flopy.utils.MfGrdFile(fname, verbose=False)
iverts, verts = grd.get_verts()
vertc = grd.get_centroids()
xc = vertc[:, 0]
yc = vertc[:, 1]
sr = flopy.utils.reference.SpatialReferenceUnstructured(xc, yc, verts, iverts, [xc.shape[0]])

In [ ]:
ibd = np.zeros((ncpl), dtype=np.int)
ibd[welcells['nodenumber']] = 1
ibd[rivcells['nodenumber']] = 2
ibd = np.ma.masked_equal(ibd, 0)
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mm = flopy.plot.ModelMap(sr=sr, ax=ax)
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)
cmap = mpl.colors.ListedColormap(['r','g',])
v = mm.plot_cvfd(verts, iverts, edgecolor='gray', a=ibd, cmap=cmap)
t = ax.set_title('Boundary Conditions')

In [ ]:
fname = os.path.join(model_ws, model_name + '.hds')
hdobj = flopy.utils.HeadFile(fname)
head = hdobj.get_data()
head.shape

In [ ]:
ilay = 2
cint = .25
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mm = flopy.plot.ModelMap(sr=sr, ax=ax)
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)
v = mm.plot_cvfd(verts, iverts, edgecolor='black', a=head[ilay, 0, :], cmap='jet')
hmin = head[ilay, 0, :].min()
hmax = head[ilay, 0, :].max()
levels = np.arange(np.floor(hmin), np.ceil(hmax) + cint, cint)
cs = mm.contour_array_cvfd(vertc, head[ilay, 0, :], colors='white', 
                           levels=levels)
plt.clabel(cs, fmt='%.1f', colors='white', fontsize=11)
cb = plt.colorbar(v, shrink=0.5)
t = ax.set_title('Model Layer {}; hmin={:6.2f}, hmax={:6.2f}'.format(ilay + 1, hmin, hmax))

### Model Cells and Vertices

In [ ]:
# zoom area
xmin, xmax = 2000, 4300
ymin, ymax = 5400, 7500

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mm = flopy.plot.ModelMap(sr=sr, ax=ax)
v = mm.plot_cvfd(verts, iverts, edgecolor='black', facecolor='none')
t = ax.set_title('Model Cells and Vertices (one-based)')
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.plot(verts[:, 0], verts[:, 1], 'bo')

for i in range(nvert):
    x = verts[i, 0]
    y = verts[i, 1]
    if xmin <= x <= xmax and ymin <= y <= ymax:
        ax.annotate(str(i + 1), verts[i, :], color='b')
        
for i in range(ncpl):
    x = vertc[i, 0]
    y = vertc[i, 1]
    if xmin <= x <= xmax and ymin <= y <= ymax:
        ax.annotate(str(i + 1), vertc[i, :], color='r')    